In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
import matplotlib.pyplot as plt
# %matplotlib nbagg
%matplotlib inline
import seaborn as sns

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
test_features['cp_type'] = test_features['cp_type'].map({'trt_cp':0, 'ctl_vehicle':1})
test_features['cp_dose'] = test_features['cp_dose'].map({'D1':0, 'D2':1})

train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_features['cp_type'] = train_features['cp_type'].map({'trt_cp':0, 'ctl_vehicle':1})
train_features['cp_dose'] = train_features['cp_dose'].map({'D1':0, 'D2':1})

train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')

Первое,небольшой анализ структуры данных.

In [ ]:
# Убедимся, что нет перескечений
set(test_features['sig_id']) & set(train_features['sig_id'])

In [ ]:
train_features[['cp_type']].value_counts()

In [ ]:
train_features[['cp_dose']].value_counts()

In [ ]:
def create_mean_std_df(df):
    means = df.mean()
    stds = df.std()
    ret_df = pd.concat([means, stds], axis=1).reset_index()
    ret_df.columns = ['index', 'mean', 'std']
    ret_df['pref'] = ret_df['index'].apply(lambda x: x.split('-')[0])
    ret_df['post'] = ret_df['index'].apply(lambda x: x.split('-')[1])
    return ret_df

def scatter(y, x=None, color= None):
    fig, ax = plt.subplots()
    un_c = color.unique()
    for i, col in enumerate(un_c):
        y_i = y[color == col]
        if type(x) != pd.core.series.Series:
            x_i = range(0 + i * (y.shape[0] - y_i.shape[0]),
                    (1 - i)*y_i.shape[0] + i * y.shape[0])
        else:
            x_i = x[color == col]
        ax.scatter(x_i, y_i, c=col, label=col,
               alpha=0.3)
        ax.set_ylabel(y.name)
        ax.legend()
        ax.grid(True)
    plt.show()

In [ ]:
tr = create_mean_std_df(train_features.iloc[:,4:])

In [ ]:
scatter(y=tr['mean'], color = tr['pref'])

In [ ]:
scatter(y = tr['std'], color = tr['pref'])

In [ ]:
scatter(y = tr['std'], x = tr['mean'], color = tr['pref'])

In [ ]:
corrMatrix = train_features.corr()

In [ ]:
corrMatrix.tail(10)

In [ ]:
plt.imshow(corrMatrix)
plt.show()

Чем ярче цвет, тем ближе к 1, темный цвет - к -1.

In [ ]:
plt.imshow(corrMatrix.sort_values(by=list(corrMatrix.columns)))
plt.show()

Первая модель, которая была построена - logreg.

In [ ]:
def mul_lab_logreg(test, train_X, train_y):
#   Шаблон для Submission
    sub = pd.DataFrame(test['sig_id'])
    
#   Пройдемся по каждой колонке и обучим одну модель логрег, выдадим pred_proba для колонки шаблона  
    col = train_y.columns.drop('sig_id')
    train_X.set_index('sig_id', inplace=True)
    df = pd.concat([train_X.iloc[:,0],
                    train_y.set_index('sig_id')], axis = 1)
    for c in tqdm(col):
        print(c)
        y = df.loc[:, c]
        clf = LogisticRegression(random_state=0, 
                         class_weight= y.mean(), 
                         n_jobs=6).fit(train_X, y)

        clf.fit(train_X, y)
        sub[c] = clf.predict_proba(test.drop('sig_id', axis=1)).T[1]
    return sub

# probs = mul_lab_logreg(test_features,train_features,train_targets_scored)

In [ ]:
# probs = mul_lab_logreg(test_features,train_features,train_targets_scored)

Далее опробован случайный лес.

In [ ]:
def score(preds, true_y):
    log_preds = np.log(preds + 0.00000001)
    log_inv_preds = np.log(1 - preds)
    total = true_y * log_preds + (1 - true_y) * log_inv_preds
    return total.sum().sum() / (preds.shape[0]*preds.shape[1])


In [ ]:
clf = RandomForestClassifier(n_estimators=15, criterion='entropy', max_depth=15, max_samples=150,
                             max_features = 0.3, verbose=1,
                            n_jobs=-1, random_state=1998, ccp_alpha=0.0)

In [ ]:
clf.fit(train_features.set_index('sig_id'), train_targets_scored.set_index('sig_id'))

In [ ]:
preds = clf.predict_proba(test_features.drop('sig_id', axis=1))

In [ ]:
sub = pd.DataFrame(test_features['sig_id'])
col = train_targets_scored.columns.drop('sig_id')

In [ ]:
col = train_targets_scored.columns.drop('sig_id')

In [ ]:
for p, c in tqdm(zip(preds, col)):
        print(c)
        sub[c] = p.T[1]

In [ ]:
plt.bar(sub.mean().index, sub.mean())

In [ ]:
plt.bar(train_targets_scored.mean().index, train_targets_scored.mean())

In [ ]:
sub.to_csv('submission.csv', index=None)

In [ ]:
submit.reset_index().to_csv('submission.csv', index=None)